## <pre>                               <b> Model Testing and Prediction

# Import all necessary libraries

In [1]:
from keras.models import Model
from keras.layers.recurrent import LSTM
from keras.layers import Dense, Input, Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from collections import Counter
import nltk
import numpy as np
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
import keras
import nltk
import numpy as np
import sklearn
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
RAND_STATE=np.random.seed(42)
BATCH_SIZE = 32
NUM_EPOCHS = 10
GLOVE_EMBEDDING_SIZE = 100
HIDDEN_UNITS = 256
MAX_INPUT_SEQ_LENGTH = 40
MAX_TARGET_SEQ_LENGTH = 40
MAX_VOCAB_SIZE = 10000
DATA_SET_NAME = '/content/drive/My Drive/Chatbot/Chatbot'
DATA_PATH = '/content/drive/My Drive/Chatbot/Chatbot/movie_lines_cleaned.txt'
GLOVE_MODEL = "/content/drive/My Drive/glove.twitter.27B.100d.txt"
WHITELIST = 'abcdefghijklmnopqrstuvwxyz1234567890?.,'
WEIGHT_FILE_PATH = '/content/drive/My Drive/Chatbot/Chatbot/word-glove-weights.h5'

In [0]:
def in_white_list(_word):
    return set(_word) <= set(WHITELIST)

In [0]:
def load_glove_vector():
    _word2embedding = {}
    file = open(GLOVE_MODEL, mode='rt', encoding='utf8')
    for line in file:
        splitLine = line.split()
        if in_white_list(splitLine[0]):
            _word2embedding[splitLine[0]] = np.array([float(val) for val in splitLine[1:]])
    file.close()
    return _word2embedding

## <i>Defining chatbot function to get the required output 

In [0]:
class CornellChatBot(object):
    model = None
    encoder_model = None
    decoder_model = None
    target_word2idx = None
    target_idx2word = None
    max_decoder_seq_length = None
    max_encoder_seq_length = None
    num_decoder_tokens = None
    word2embedding = None

    def __init__(self):
        self.word2embedding = load_glove_vector()
        print(len(self.word2embedding))
        print(self.word2embedding['start'])

        self.target_word2idx = np.load( DATA_SET_NAME + '/word-glove-target-word2idx.npy',allow_pickle=True).item()
        self.target_idx2word = np.load( DATA_SET_NAME + '/word-glove-target-idx2word.npy',allow_pickle=True).item()
        context = np.load( DATA_SET_NAME + '/word-glove-context.npy',allow_pickle=True).item()
        self.max_encoder_seq_length = context['encoder_max_seq_length']
        self.max_decoder_seq_length = context['decoder_max_seq_length']
        self.num_decoder_tokens = context['num_decoder_tokens']

        encoder_inputs = Input(shape=(None, GLOVE_EMBEDDING_SIZE), name='encoder_inputs')
        encoder_lstm = LSTM(units=HIDDEN_UNITS, return_state=True, name="encoder_lstm")
        encoder_outputs, encoder_state_h, encoder_state_c = encoder_lstm(encoder_inputs)
        encoder_states = [encoder_state_h, encoder_state_c]

        decoder_inputs = Input(shape=(None, GLOVE_EMBEDDING_SIZE), name='decoder_inputs')
        decoder_lstm = LSTM(units=HIDDEN_UNITS, return_sequences=True, return_state=True, name='decoder_lstm')
        decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
        decoder_dense = Dense(self.num_decoder_tokens, activation='softmax', name='decoder_dense')
        decoder_outputs = decoder_dense(decoder_outputs)

        self.model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


        self.model.load_weights(WEIGHT_FILE_PATH)
        self.model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

        self.encoder_model = Model(encoder_inputs, encoder_states)

        decoder_state_inputs = [Input(shape=(HIDDEN_UNITS,)), Input(shape=(HIDDEN_UNITS,))]
        decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_state_inputs)
        decoder_states = [state_h, state_c]
        decoder_outputs = decoder_dense(decoder_outputs)
        self.decoder_model = Model([decoder_inputs] + decoder_state_inputs, [decoder_outputs] + decoder_states)

    def reply(self, input_text):
        input_seq = []
        input_emb = []
        for word in nltk.word_tokenize(input_text.lower()):
            if not in_white_list(word):
                continue
            emb = np.zeros(shape=GLOVE_EMBEDDING_SIZE)
            if word in self.word2embedding:
                emb = self.word2embedding[word]
            input_emb.append(emb)
        input_seq.append(input_emb)
        input_seq = pad_sequences(input_seq, self.max_encoder_seq_length)
        states_value = self.encoder_model.predict(input_seq)
        target_seq = np.zeros((1, 1, GLOVE_EMBEDDING_SIZE))
        target_seq[0, 0, :] = self.word2embedding['start']
        target_text = ''
        target_text_len = 0
        terminated = False
        while not terminated:
            output_tokens, h, c = self.decoder_model.predict([target_seq] + states_value)

            sample_token_idx = np.argmax(output_tokens[0, -1, :])
            sample_word = self.target_idx2word[sample_token_idx]
            target_text_len += 1

            if sample_word != 'start' and sample_word != 'end':
                target_text += ' ' + sample_word

            if sample_word == 'end' or target_text_len >= self.max_decoder_seq_length:
                terminated = True

            target_seq = np.zeros((1, 1, GLOVE_EMBEDDING_SIZE))
            if sample_word in self.word2embedding:
                target_seq[0, 0, :] = self.word2embedding[sample_word]

            states_value = [h, c]
        return target_text.strip()

    def test_run(self):
        print(self.reply('What do you know'))

## Creating object for class

In [7]:
bot = CornellChatBot()

559648
[-0.17989   0.22297   0.47938  -0.71227  -0.45818   1.0285    0.32394
 -0.060409 -0.37064   0.3051   -0.14261  -0.56449  -4.5301    0.54817
 -0.85281  -0.086907 -0.28587   0.86288  -0.28724  -0.65113  -0.97384
  0.11036  -0.05808  -0.034859 -0.36309   0.19478   0.17636  -0.32154
 -0.22864  -0.11961  -0.044675  0.54424  -0.25474   0.21692   0.5004
  0.21677   0.33958  -0.27821   0.58674   0.013013 -0.98293   0.5214
  0.11687  -0.10702   0.1903    0.25038  -0.24482  -0.068194 -0.23054
  0.24936   0.081091 -0.71015   0.050871 -0.16209   0.49785  -0.44498
 -0.79807  -0.1008    0.80597   0.18716  -0.65218  -0.27916   0.23074
 -0.35599  -0.18894   0.36532   0.74004  -0.29412   0.90441   0.067676
 -0.19106   0.59315   0.058992  0.53448   0.32551   0.060201  0.28332
  0.026973 -0.079146 -0.40832   1.3507   -0.1911   -0.23131  -0.37369
  0.32181   0.10459  -0.11756   0.028256  0.27408  -0.289    -0.21644
  0.17697  -0.23683   0.15782  -0.22889   0.26629  -0.28217   0.29003
 -0.032464 -0.

### <b>Testing the bot

In [8]:
bot.test_run()

i do n't


## Using Flask to interact with the bot

In [0]:
chatbot_html = """
<style type="text/css">
#log p { margin: 5px; font-family: sans-serif; align:center;}</style>
<div id="log"
     style="box-sizing: border-box;
            width: 600px;
            height: 32em;
            border: 1px grey solid;
            padding: 2px;
            overflow: scroll;">
</div>
<input type="text" id="typehere" placeholder="type here!"
       style="box-sizing: border-box;
              width: 600px;
              margin-top: 5px;">
<script>
function paraWithText(t) {
    let tn = document.createTextNode(t);
    let ptag = document.createElement('p');
    ptag.appendChild(tn);
    return ptag;
}
document.querySelector('#typehere').onchange = async function() {
    let inputField = document.querySelector('#typehere');
    let val = inputField.value;
    inputField.value = "";
    let resp = await getResp(val);
    let objDiv = document.getElementById("log");
    objDiv.appendChild(paraWithText('😀: ' + val));
    objDiv.appendChild(paraWithText('🤖: ' + resp));
    objDiv.scrollTop = objDiv.scrollHeight;
};
async function colabGetResp(val) {
    let resp = await google.colab.kernel.invokeFunction(
        'notebook.get_response', [val], {});
    return resp.data['application/json']['result'];
}
async function webGetResp(val) {
    let resp = await fetch("/response.json?sentence=" + 
        encodeURIComponent(val));
    let data = await resp.json();
    return data['result'];
}
</script>
"""

## Creating a localhost to run chatbot
             OR
### Go below to use Tkinter

In [10]:
from flask import Flask, request, jsonify
app = Flask(__name__)
@app.route("/response.json")
def response():
    sentence = request.args['sentence']
    return jsonify(
        {'result': bot.reply(sentence)})
@app.route("/")
def home():
    return chatbot_html + "<script>let getResp = webGetResp;</script>"
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


## Using Tkinter to create a chatbot like visuals



    .

In [0]:
#@title Default title text
#Creating GUI with tkinter
import tkinter
from tkinter import *

def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        
        ChatLog.insert(END, "YOU: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = model.reply(msg)
        ChatLog.insert(END, "BOT: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)


base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="blue", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial", borderwidth=5)
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()
